# Stanford RNA 3D Folding
- *notify  :  The work is based on **PyTorch**. As a beginner participating in this competition,its my great honor to have ur comments and encouragement. Criticism and corrections are also welcome.*
- *Author  :  **Chengzhi Jiang***

In [1]:
import numpy as np
import pandas as pd

## Dataset Overview

In [2]:
train_labels = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/train_labels.csv")
train_sequence = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/train_sequences.csv")
val_labels = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/validation_labels.csv")
val_sequence = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/validation_sequences.csv")
test_sequence = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/test_sequences.csv")

print("Train Seq: " + str(train_sequence.shape))
print("Train Label: " + str(train_labels.shape))
print("Validation Seq: " + str(val_sequence.shape))
print("Validation Label: " + str(val_labels.shape))
print("Test: "+str(test_sequence.shape))

Train Seq: (844, 5)
Train Label: (137095, 6)
Validation Seq: (12, 5)
Validation Label: (2515, 123)
Test: (12, 5)


In [3]:
train_labels.head()

,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,1.827,-20.136000,11.793


In [4]:
train_sequence.head()

,target_id,sequence,temporal_cutoff,description,all_sequences
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...
1,1RNK_A,GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU,1995-02-27,THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES...,>1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUG...
2,1RHT_A,GGGACUGACGAUCACGCAGUCUAU,1995-06-03,24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE F...,>1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*...
3,1HLX_A,GGGAUAACUUCGGUUGUCCC,1995-09-15,P1 HELIX NUCLEIC ACIDS (DNA/RNA) RIBONUCLEIC ACID,>1HLX_1|Chain A|RNA (5'-R(*GP*GP*GP*AP*UP*AP*A...
4,1HMH_E,GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU,1995-12-07,THREE-DIMENSIONAL STRUCTURE OF A HAMMERHEAD RI...,">1HMH_1|Chains A, C, E|HAMMERHEAD RIBOZYME-RNA..."


In [5]:
val_labels.head()

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_37,x_38,y_38,z_38,x_39,y_39,z_39,x_40,y_40,z_40
0,R1107_1,G,1,-5.499,8.520000,8.605000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
1,R1107_2,G,2,-5.826,10.453000,14.010000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
2,R1107_3,G,3,-5.849,14.768000,17.584999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
3,R1107_4,G,4,-5.784,19.985001,18.666000,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18
4,R1107_5,G,5,-5.755,25.533001,17.132999,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,...,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18,-1.000000e+18


## Preprocessing

In [6]:
seq_dict = {'A': 1, 'C': 2, 'G': 3, 'U': 4}
def seq_map(seq):
    return [seq_dict.get(char, 0) for char in seq]

train_sequence['encoded_seq'] = train_sequence['sequence'].apply(seq_map)
test_sequence['encoded_seq'] = test_sequence['sequence'].apply(seq_map)
val_sequence['encoded_seq'] = val_sequence['sequence'].apply(seq_map)

In [7]:
from collections import defaultdict

def generate_label_coord(frame):
    output = defaultdict(list)

    frame['label'] = frame.ID.str.rsplit('_', n=1, expand=True).iloc[:, 0] #将frame中label中的内容修改下,从右边开始去掉一次

    for _, row in frame.iterrows():
        label = row['label']
        resid = row['resid']     #residual id 残基序号(索引)  每个核苷酸的位置由5'端 -> 3'端 一次编号

        coord = np.array((row['x_1'], row['y_1'], row['z_1']), dtype=np.float32)
        output[label].append((resid, coord))

    for key, value in output.items():  #这里如果不用item() 只能遍历key
        coords = np.stack([c for r, c in value])
        masks = np.isnan(coords) | np.isclose(coords, -1.0000e+18)  #接近阈值为True:-1.0000e+18 

        coords[masks] = 0.  #无效坐标位置置0

        output[key] = {
            'coords': coords, #得到修正后的坐标矩阵
            'masks': ~masks,  #标记有效坐标
        }

    return output

train_stacked_coords = generate_label_coord(train_labels)
val_stacked_coords = generate_label_coord(val_labels)

train_stacked_coords[list(train_stacked_coords.keys())[0]]  #获取list中第一个key的对应coords坐标信息以及mask掩码矩阵

{'coords': array([[ 13.76 , -25.974,   0.102],
        [  9.31 , -29.638,   2.669],
        [  5.529, -27.813,   5.878],
        [  2.678, -24.901,   9.793],
        [  1.827, -20.136,  11.793],
        [  2.04 , -14.908,  11.771],
        [  1.107, -11.513,   7.517],
        [  2.991,  -6.406,   4.783],
        [  0.896,  -1.193,   7.608],
        [  0.228,   2.646,   9.128],
        [  4.329,   2.718,   4.804],
        [  5.165,   4.792,  -0.914],
        [  2.61 ,   9.495,  -2.308],
        [  1.174,  13.829,   0.201],
        [  1.58 ,  20.115,   3.76 ],
        [ -1.575,  16.928,   5.897],
        [ -6.051,  14.762,   5.224],
        [ -5.554,  10.415,   4.309],
        [ -3.107,   6.405,   2.12 ],
        [ -1.41 ,   3.335,  -2.655],
        [  1.866,  -0.716,  -4.333],
        [  3.655,  -4.444,  -2.485],
        [  5.314,  -7.656,   1.13 ],
        [  7.931,  -9.528,   5.781],
        [  8.735, -12.648,  10.025],
        [  9.108, -17.296,  13.021],
        [  8.897, -22.606,  

## Create Dataset for CNN

In [8]:
def generate_dataset(seq, stacked_coords):
    X, y, my, tids = [], [], [], []

    for idx, row in seq.iterrows():  #遍历 DataFrame 的每一行，逐行处理
        tid = row['target_id']
        if tid in stacked_coords:      #sequence中的target_id 在 label.csv中存在的话
            X.append(row['encoded_seq'])
            y.append(stacked_coords[tid]['coords'])
            my.append(stacked_coords[tid]['masks'])  #掩码列表
            tids.append(tid)   # 用来跟踪样本来源

    return X, y, my, tids

train_X, train_y, train_my, train_tids = generate_dataset(train_sequence, train_stacked_coords)
val_X, val_y, val_my, val_tids = generate_dataset(val_sequence, val_stacked_coords)

In [9]:
import torch
from torch.nn.utils.rnn import pad_sequence #pad_sequence 函数的主要功能是对多个不同长度的序列进行填充（padding），使它们的长度达到一致，从而可以将这些序列组合成一个批量（batch）数据，方便输入到神经网络中进行处理。在自然语言处理、语音识别、RNA 序列分析等涉及序列数据的任务中，由于不同样本的序列长度往往不同，所以需要使用该函数进行长度统一。

def pad_sequences_torch(sequences, max_len, padding='post', value=0):  #post是在序列末尾填充pre是在前面填充（生物学上，末尾填充可以避免截断头部特征）
    padded = pad_sequence([
        torch.cat([seq, torch.full((max_len - len(seq),), value, dtype=seq.dtype)])  #torch.full()生成固定填充值的张量((x,)填充长度为x,填充值,dtype)
        if len(seq) < max_len else seq[:max_len]
        for seq in sequences
    ], batch_first=True, padding_value=value)  # batch_first:指定张量的批量维度为第一维 默认是(seq_len, batch_size, features)。**batch_first=True**：(batch_size, seq_len, features)  这里就可以不用permute了
    return padded

max_len = max(len(seq) for seq in train_X)

train_X_pad = pad_sequences_torch([torch.tensor(x) for x in train_X], max_len)
val_X_pad = pad_sequences_torch([torch.tensor(x) for x in val_X], max_len)
test_X = test_sequence['encoded_seq'].tolist()    #test还是Seires,需要转化成列表.
test_X_pad = pad_sequences_torch([torch.tensor(x) for x in test_sequence['encoded_seq'].tolist()], max_len)

train_X_pad.shape

torch.Size([844, 4298])

In [10]:
import torch.nn.functional as F

def pad_coords_torch(coords, max_len):
    L = coords.size(0)
    if L < max_len:
        pad = (0, 0, 0, max_len - L)  # (left, right, top, bottom)
        return F.pad(coords, pad, "constant", 0)        #pad()针对已有张量修改而full()是创建新张量    constant:使用固定值填充
    else:
        return coords[:max_len]

train_y_pad = torch.stack([pad_coords_torch(torch.tensor(y), max_len) for y in train_y])
val_y_pad = torch.stack([pad_coords_torch(torch.tensor(y), max_len) for y in val_y])

train_my_pad = torch.stack([pad_coords_torch(torch.tensor(my), max_len) for my in train_my])
val_my_pad = torch.stack([pad_coords_torch(torch.tensor(my), max_len) for my in val_my])

train_y_pad.shape

torch.Size([844, 4298, 3])

## CNN

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl           #PyTorch Lightning 是一个轻量级的 PyTorch 包装器，它通过提供高级的抽象和标准化的训练流程，帮助用户更高效地组织和训练 PyTorch 模型，减少了大量重复的样板代码
from pytorch_lightning.callbacks import EarlyStopping

class CNN(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # Embedding layer    嵌入层的主要作用是将离散的整数索引映射为连续的向量表示。在自然语言处理、生物信息学等领域中，经常需要将离散的符号（如单词、碱基等）转换为连续的向量，以便于神经网络进行处理。
        self.embedding = nn.Embedding(
            num_embeddings=config['seq_mapping_size'],
            embedding_dim=config['embedding_dim'],
            padding_idx=0  #可选参数，用于指定填充位置的索引。当输入序列长度不一致时，通常会使用填充操作将序列填充到相同的长度，填充位置的索引可以通过 padding_idx 指定，填充位置对应的嵌入向量通常会被设置为全零向量
        )
        
        # First Conv Block
        self.conv1 = nn.Conv1d(
            in_channels=config['embedding_dim'],
            out_channels=config['num_filters'],
            kernel_size=config['kernel_size'],
            padding='same'  #保持序列长度不变
        )
        self.bn1 = nn.BatchNorm1d(config['num_filters'])
        self.drop1 = nn.Dropout(config['drop_rate'])
        
        # Second Conv Block
        self.conv2 = nn.Conv1d(
            in_channels=config['num_filters'],
            out_channels=config['num_filters'],
            kernel_size=config['kernel_size'],
            padding='same'
        )
        self.shortcut = nn.Conv1d(config['num_filters'], config['num_filters'], kernel_size=1)
        
        self.bn2 = nn.BatchNorm1d(config['num_filters']) #在深度神经网络的训练过程中，每一层的输入分布会随着前面层的参数更新而发生变化，这被称为内部协变量偏移（Internal Covariate Shift）。内部协变量偏移会使得后续层需要不断适应输入分布的变化，从而减慢模型的收敛速度。nn.BatchNorm1d 通过对每一批次的输入数据进行归一化处理，使得每一层的输入数据的均值接近 0，方差接近 1，从而减少了内部协变量偏移的影响。
        self.drop2 = nn.Dropout(config['drop_rate'])
        
        
        # Final Prediction Layer
        self.final_conv = nn.Conv1d(
            in_channels=config['num_filters'],
            out_channels=3,  # x , y , z
            kernel_size=1,    #为啥是1 ->不改变特征图尺寸1x1卷积核的空间感受野为1，在默认步长（stride=1）和填充（padding='same'）时，不会改变特征图的高度（H）和宽度（W），仅调整通道数（C）。应用场景：在RNA坐标预测任务中，需保持每个核苷酸残基的位置对齐（即序列长度L不变），确保输出坐标与输入序列一一对应
            padding='same'
        )
        
        self.loss_fn = nn.MSELoss(reduction='none')  #none:不进行任何聚合，直接返回每个位置（或元素）的独立损失值-适用与RNA预测  若是mean适用于回归任务 sum适用于回归任务的总误差计算；交叉熵损失中加权求和处理类别不平衡

    def forward(self, x):
        # Embedding
        x = self.embedding(x)  # (batch, seq_len, embedding_dim)
        x = x.permute(0, 2, 1)  # (batch, embedding_dim, seq_len)   - Conv1d要求输入张量格式为(batch, channels, sequence_length)  这个permute是torch.Tensor里面的方法
        
        # First Conv Block
        x = F.silu(self.conv1(x))  #激活函数引入非线性  通道数变为num_filters
        x = self.bn1(x)
        x = self.drop1(x)
        
        # Second Conv Block
        residual = self.shortcut(x)
        x = F.silu(self.conv2(x)+residual)  # Swish = x * sigmoid(x)
        x = self.bn2(x)
        x = self.drop2(x)

        # Final Prediction
        x = self.final_conv(x)
        x = x.permute(0, 2, 1)  # (batch, seq_len, 3)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y, my = batch    #batch 包含数据 x、标签 y 和掩码 my
        y_hat = self(x)  #也就是forawrd(x)
        loss = self.loss_fn(y_hat, y)
        loss = (my * loss).mean()   ##掩码过滤：通过逐元素乘法 my * loss 过滤无效位置（掩码值为 0 的位置损失归零）   生物学意义：在 RNA 结构预测中，掩码常用于忽略填充的无效核苷酸位置，确保模型仅优化有效区域的坐标

        
        self.log('train_loss', loss, prog_bar=True)  # 记录训练损失，prog_bar=True 显示在进度条
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y, my = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        loss = (my * loss).mean()  

        self.log('val_loss', loss, prog_bar=True)  
        return loss
    
    def configure_optimizers(self):
        # return torch.optim.Adam(self.parameters())
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3, weight_decay=0.01)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=2
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"},
        }


config = {
    'seq_mapping_size': max(seq_dict.values()) + 1,
    'embedding_dim': 16,
    'num_filters':32,
    'kernel_size': 3,
    'drop_rate': 0.4,
    'weight_decay': 0.0035,
    'train_epochs': 550,
    'batch_size': 16
}


train_dataset = TensorDataset(train_X_pad, train_y_pad, train_my_pad.long())
val_dataset = TensorDataset(val_X_pad, val_y_pad, val_my_pad.long())

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True,num_workers = 3)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'],num_workers = 3)


model = CNN(config)
early_stop = EarlyStopping(monitor='val_loss', patience=3, mode='min')

trainer = pl.Trainer(
    max_epochs=config['train_epochs'],
    callbacks=[early_stop],
    accelerator='auto',
    precision='16-mixed',  # 启用混合精度 -  自动切换FP16和FP32   - 技术原理前向传播：使用 FP16 计算，减少显存占用和计算时间。反向传播：仍用 FP16 计算梯度，但通过梯度缩放（Gradient Scaling）防止数值下溢。参数更新：最终参数用 FP32 存储，避免精度损失。
    accumulate_grad_batches=4, #模拟更大batch size，提升训练稳定性.过累积实现等效batch_size=128
    gradient_clip_val=0.5,  # 梯度裁剪 由于加了梯度累计需要加梯度裁剪防止梯度爆炸导致模型无法收敛
    enable_progress_bar=True,
    log_every_n_steps=1,
)
trainer.fit(model, train_loader, val_loader)


# 训练后保存模型
torch.save(model.state_dict(), "rna_model.pth")

# 测试时重新加载
model = CNN(config)
# 安全加载：仅加载参数，忽略其他对象
state_dict = torch.load(
    "rna_model.pth",
    weights_only=True,  # 正确位置
    map_location='cpu'  # 可选：指定设备
)
model.load_state_dict(state_dict)


test_tensor = torch.LongTensor(test_X_pad)
model.eval()
with torch.no_grad():
    pred = model(test_tensor).cpu().numpy()

print(pred.shape)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

(12, 4298, 3)


## Submission
- the target 3 dimensional coordinates are duplicated 5 times

In [ ]:
rows = []

for idx, row in test_sequence.iterrows():
    target_id = row['target_id']
    coords = pred[idx]
    seq_length = len(row['encoded_seq'])
    coords = coords[:seq_length, :]

    for i in range(seq_length):
        x, y, z = coords[i, :]
        rows.append(
            {
                'ID': f"{target_id}_{i+1}",
                'resname': row['sequence'][i],
                'resid': i+1,
                 **{f"x_{j+1}": x for j in range(5)},
                 **{f"y_{j+1}": y for j in range(5)},
                 **{f"z_{j+1}": z for j in range(5)}
            }
        )

submission = pd.DataFrame(rows)
submission.to_csv("submission.csv", index=False)

submission.shape